In [1]:
# Create a folder to store csv files
import pathlib
pathlib.Path('./Punctate').mkdir()

In [2]:
import numpy as np
import random as rd
import pandas as pd

In [22]:
def punc1(gamma, alpha, end_states, rewards, transitions, v_state, state_list, action_list, RPE_list):
    time_step = 1
    current_state = 0
    timestep_list = []
    not_end = True
    
    while not_end:
        if current_state in end_states:
            not_end = False
            break
        
        else:
            # Get reward
            reward = rewards[current_state]
            
            # Determine the next state
            # Random chance of choosing any of the subsequent states(?)
            next_move = np.random.randint(len(transitions[current_state]))
            next_state = transitions[current_state][next_move]
            
            # calculate RPE and update weights and state values
            if current_state in end_states: # at the goal state
                delta = reward + 0 - v_state[current_state]
            else:
                delta = reward + gamma*v_state[next_state] - v_state[current_state]
            
            # update state value
            v_state[current_state] += alpha * delta
            
            state_list.append(current_state+1)
            action_list.append(next_state)
            RPE_list.append(delta)
            timestep_list.append(time_step)

            # Move to the next state
            current_state = next_state
            
            time_step += 1

    return v_state, state_list, action_list, RPE_list, timestep_list

In [15]:
# function for multi episodes
def punc2(epi_num, gamma, alpha, end_states, rewards, transitions, v_state, state_list, action_list, RPE_list, epi_num_list):
    epi_length = []
    for k in range(epi_num):
        c_v_state, c_state_list, c_action_list, c_RPE_list, timestep_list = \
        punc1(gamma, alpha, end_states, rewards, transitions, v_state, state_list, action_list, RPE_list)
        
        for j in range(len(timestep_list)):
            epi_num_list.append(k+1)
                
        for j in range(len(timestep_list)):
            epi_length.append(k+1)
        
        v_state = c_v_state
        state_list = c_state_list
        action_list = c_action_list
        RPE_list = c_RPE_list
        
    return c_v_state, c_state_list, c_action_list, c_RPE_list, epi_num_list, epi_length

In [16]:
# function for multi simulations
def punc3(sim_num, epi_num, gamma, alpha, end_states, rewards, transitions, state_list, action_list, RPE_list, epi_num_list):
    sim_num_list = []
    
    for t in range(sim_num):
        v_state = rewards
        
        c_v_state, c_state_list, c_action_list, c_RPE_list, c_epi_num_list, epi_length = \
        punc2(epi_num, gamma, alpha, end_states, rewards, transitions, v_state, state_list, action_list, RPE_list, epi_num_list)
        
        for u in range(len(epi_length)):
            sim_num_list.append(t+1)
        
        state_list = c_state_list
        action_list = c_action_list
        RPE_list = c_RPE_list
        epi_num_list = c_epi_num_list
    
    return c_v_state, c_state_list, c_action_list, c_RPE_list, c_epi_num_list, sim_num_list

In [43]:
sim_num = 1
epi_num = 1000
alpha = 0.50
gamma = 0.95
state_list = []
action_list = []
RPE_list = []
epi_num_list = []

end_states = [3, 4, 5]
rewards = [0, 0, 0, 15, 0, 30]
transitions = [[1, 2], [3, 4], [4, 5]]

rl = punc3(sim_num, epi_num, gamma, alpha, end_states, rewards, transitions, state_list, action_list, RPE_list, epi_num_list)

In [44]:
print(rl)

([668146.6968750008, 1645.875, 3933.0, 15, 0, 30], [1, 3, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 

In [ ]:
# Simulations with various parameters
import numpy as np
import random as rd
import pandas as pd

seed_list = [22, 76, 50, 57, 30, 55, 33, 54,  0]
index = 0

for gamma in [0.95, 0.97, 0.99]:
    for stay_prob in [0.50, 0.75, 0.90]:
        
        rd.seed(seed_list[index])
        
        # set constant variables
        sim_num = 100
        epi_num = 200
        alpha = 0.50
        state_n = 10
        state_list = []
        action_list = []
        RPE_list = []
        epi_num_list = []
        
        # simulation
        rl = punc3(sim_num, epi_num, gamma, alpha, state_n, stay_prob, state_list,
                  action_list, RPE_list, epi_num_list)
        
        # create dataframe and convert it to csv
        result = pd.DataFrame({'Simulation': rl[5], 'Episode': rl[4], 'State': rl[1],
                              'Action': rl[2], 'RPE': rl[3]})
        result.to_csv('./Punctate/g{:.0f}_s{:.0f}_{:.0f}states.csv'.format(100*gamma, 100*stay_prob, state_n))
        index += 1